In [ ]:
import healpy as hp
import numpy as np
import pickle
import h5py
import altair as alt
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [ ]:
cd ..

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
telescope="SAT"
nside=4096 if telescope == "LAT" else 512


In [ ]:
from astropy.table import QTable

In [ ]:
s4 = QTable.read("instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac")

In [ ]:
rows = s4[s4["telescope"] == telescope]

In [ ]:
channels = list(rows["band"])

In [ ]:
channels

In [ ]:
freqs = rows["center_frequency"]

In [ ]:
print(freqs)

In [ ]:
component = "combined_foregrounds"
print(component)
for ch in channels:
    print(ch)
    lat_chile = telescope == "LAT" and "P" not in ch
    site = "chile" if lat_chile else "pole"
    m = hp.read_map(
   "output/foregrounds/" + \
        f"{telescope}-{ch}_{site}/cmbs4_KCMB_{telescope}-{ch}_{site}_nside{nside}_1_of_1.fits")
    m[m != hp.UNSEEN] *= 1e6
    m_input = hp.read_map(
        f"../in/output/{nside}/combined_foregrounds/0000/cmbs4_combined_foregrounds_uKCMB_{telescope}-{ch}_nside{nside}_0000.fits")
    m_input[m == hp.UNSEEN] = hp.UNSEEN
    scale = 3e-1*1e3
    if site == "chile":
        hp.mollview(m_input, xsize=3800, min=0, max=scale, unit="muK", title=f"{ch} Input")
    else:
        hp.gnomview(m_input, rot=(40,-55), xsize=3800, ysize=2400, min=0, max=scale, unit="muK", title=f"{ch} Input")
    plt.savefig(f"plot_gnomview/foreground_map_{ch}_input.png")
    if site == "chile":
        hp.mollview(m, xsize=3800, min=0, max=scale, unit="muK", title=f"{ch} Output")
    else:
        hp.gnomview(m, rot=(40,-55), xsize=3800, ysize=2400, min=0, max=scale, unit="muK", title=f"{ch} Output")
    plt.savefig(f"plot_gnomview/foreground_map_{ch}_output.png")
del m, m_input

In [ ]:
len(channels)

In [ ]:
component = "combined_foregrounds"
print(component)

for ch in channels:
    print(ch)
    lat_chile = telescope == "LAT" and "P" not in ch
    site = "chile" if lat_chile else "pole"
    mm = hp.read_map(
   "output/foregrounds/" + \
        f"{telescope}-{ch}_{site}/cmbs4_KCMB_{telescope}-{ch}_{site}_nside{nside}_1_of_1.fits", (1,2))
    mm[mm == hp.UNSEEN] = np.nan
    m = np.sqrt(mm[0]**2+mm[1]**2)
    del mm
    m *= 1e6
    m[np.isnan(m)] = hp.UNSEEN
    
    mm_input = hp.ma(hp.read_map(
        f"../in/output/{nside}/combined_foregrounds/0000/cmbs4_combined_foregrounds_uKCMB_{telescope}-{ch}_nside{nside}_0000.fits", (1,2)))
    m_input = np.sqrt(mm_input[0]**2+mm_input[1]**2)
    del mm_input
    m_input[m == hp.UNSEEN] = hp.UNSEEN
    scale = 1e-2*1e3
    if site=="chile":
        hp.mollview(m_input, xsize=3800, min=0, max=scale, unit="muK", title=f"{ch} Input")
    else:
    
        hp.gnomview(m_input, rot=(40,-55), xsize=3800, ysize=2400, min=0, max=scale, unit="muK", title=f"{ch} Input")
    plt.savefig(f"plot_gnomview/foreground_map_P_{ch}_input.png")
    if site=="chile":
        hp.mollview(m, xsize=3800, min=0, max=scale, unit="muK", title=f"{ch} Output")
    else:
        hp.gnomview(m, rot=(40,-55), xsize=3800, ysize=2400, min=0, max=scale, unit="muK", title=f"{ch} Output")
    plt.savefig(f"plot_gnomview/foreground_map_P_{ch}_output.png")

In [ ]:
for ch in channels:
    if "P" in ch:
        print(f"![](foreground_map_{ch}_input.png) ![](foreground_map_{ch}_output.png)")